In [ ]:
print('Gajraj')

In [ ]:
!pip install av clip_client jina gradio

In [ ]:
import gradio as gr
from docarray import Document,DocumentArray
from numpy import rot90
from clip_client import Client

In [30]:
def prepare_videos_to_tensor(search_query,video_info):
    video_data = Document(uri=video_info).load_uri_to_video_tensor()
    keyframe_indices = video_data.tags['keyframe_indices']
    keyframes = DocumentArray()
    for idx in range(0, len(keyframe_indices) - 1):
        keyframe_number = keyframe_indices[idx]
        keyframe_tensor = rot90(video_data.tensor[keyframe_number], -1)
        clip_indices = {
            'start': str(keyframe_number),
            'end': str(keyframe_indices[idx + 1]),
        }
        keyframe = Document(tags=clip_indices, tensor=keyframe_tensor)
        keyframes.append(keyframe)
    
    return keyframes , video_data 

def get_processor():
    server_url = "grpcs://api.clip.jina.ai:2096"
    jina_auth_token = "ef8dd06701869bc93d1849de103617ae"
    client = Client(server_url, credential={"Authorization": jina_auth_token})  
    return client

def query(search_text,keyframes,search_engine):
    query = Document(text=search_text, matches=keyframes)
    return search_engine.rank([query])[0]    

def show_results(ranked_result,video_data):
    match = ranked_result.matches[0]
    start_frame = int(match.tags['start'])
    end_frame = int(match.tags['end'])
    clip_data = video_data.tensor[start_frame:end_frame] 
    Document(tensor=clip_data).save_video_tensor_to_file("match.mp4")
    return 'match.mp4'

def main(param1,search_query,video_info):
    print(f'{search_query} , {video_info}')
    keyframes,video_data=prepare_videos_to_tensor(search_query,video_info)
    client=get_processor()
    ranked_result=query(search_text=search_query,keyframes=keyframes,search_engine=client)
    print(ranked_result.matches[0].tags)
    return show_results(ranked_result=ranked_result, video_data= video_data)

In [ ]:
main('','car on the road opposite office','/tmp/baby5v6_atdv.mp4')

In [31]:
# %%writefile ./server.py
import gradio as gr
from docarray import Document,DocumentArray
from numpy import rot90
from clip_client import Client

render=f"""
        <h3 color='red'> Welcome to the world of nightmare </h3> 
        """

def main(param1,search_query,video_info):
    print(f'{search_query} , {video_info}')
    keyframes,video_data=prepare_videos_to_tensor(search_query,video_info)
    client=get_processor()
    ranked_result=query(search_text=search_query,keyframes=keyframes,search_engine=client)
    print(ranked_result.matches[0].tags)
    return show_results(ranked_result=ranked_result, video_data= video_data)

with gr.Blocks() as solution:
    title=gr.Markdown(render)
    lbl=gr.Textbox(placeholder="Enter your search here...")
    video=gr.Video(elem_id='input_video_source')
    video_output=gr.Video(elem_id='output_video_source')
    btn=gr.Button('Fire')
    btn.click(fn=main,inputs=[title,lbl,video],outputs=[video_output])

solution.launch(share=True)

Running on local URL:  http://127.0.0.1:7861

Setting up a public link... we have recently upgraded the way public links are generated. If you encounter any problems, please report the issue and downgrade to gradio version 3.13.0
.
Running on public URL: https://5fb0a838-9c5b-4a7e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


A women with wearing goggles  , /tmp/babye5apwn3a.mp4



{'end': '1070', 'start': '918'}
